## Apprentissage en contexte

- **Objectif :** Évaluer Llama 3 (3B ou 8B) sans mettre à jour les poids (Inférence seule).
- **Expériences :**
    - **0-shot :** Demander au modèle de classer sans exemples.
    - **Few-shot :** Fournir quelques exemples de prémisse/hypothèse/étiquette avant de demander la prédiction.
    - **Chain-of-Thought (CoT) :** Prompter le modèle pour qu'il explique pourquoi il a choisi une étiquette. Cela répond à l'objectif pédagogique de compréhension du CoT.

### Chargement des données

In [1]:
import polars as pl

df_train = pl.read_csv(
    "https://raw.githubusercontent.com/mathisdrn/nlp-representation-learning/refs/heads/main/data/nli_fr_train.tsv",
    separator="\t",
    new_columns=["premise", "hypothesis", "label"],
)
df_val = pl.read_csv(
    "https://raw.githubusercontent.com/mathisdrn/nlp-representation-learning/refs/heads/main/data/nli_fr_test.tsv",
    separator="\t",
    new_columns=["premise", "hypothesis", "label"],
)
label_mapping = {
    "neutral": "Neutre",
    "entailment": "Conséquence",
    "contradiction": "Contradiction",
}
df_train = df_train.with_columns(pl.col("label").replace(label_mapping))
df_val = df_val.with_columns(pl.col("label").replace(label_mapping))
df_train, df_val

(shape: (5_010, 3)
 ┌─────────────────────────────────┬─────────────────────────────────┬───────────────┐
 │ premise                         ┆ hypothesis                      ┆ label         │
 │ ---                             ┆ ---                             ┆ ---           │
 │ str                             ┆ str                             ┆ str           │
 ╞═════════════════════════════════╪═════════════════════════════════╪═══════════════╡
 │ Eh bien, je ne pensais même pa… ┆ Je ne lui ai pas parlé de nouv… ┆ Contradiction │
 │ Eh bien, je ne pensais même pa… ┆ J'étais si contrarié que je co… ┆ Conséquence   │
 │ Eh bien, je ne pensais même pa… ┆ Nous avons eu une grande discu… ┆ Neutre        │
 │ Et je pensais que c'était un p… ┆ Je n'avais pas conscience que … ┆ Neutre        │
 │ Et je pensais que c'était un p… ┆ J'avais l'impression que j'éta… ┆ Conséquence   │
 │ …                               ┆ …                               ┆ …             │
 │ Davidson ne devrait p

### Définition des prompts

In [2]:
system_message = (
    "Tu es un expert en logique et linguistique française. "
    "Ta tâche est de déterminer la relation entre une 'Prémisse' et une 'Hypothèse'.\n\n"
    "Utilise strictement les définitions suivantes :\n"
    "- Conséquence : L'hypothèse découle nécessairement de la prémisse.\n"
    "- Contradiction : L'hypothèse est impossible si la prémisse est vraie.\n"
    "- Neutre : La vérité de la prémisse ne permet pas de déterminer celle de l'hypothèse.\n\n"
)

zero_shot_prompt = (
    system_message
    + "Réponds uniquement par le nom de la relation (Conséquence, Contradiction, ou Neutre)."
)

# Define the template structure
example_template = pl.format(
    """<exemple>
    <premise>{}</premise>
    <hypothesis>{}</hypothesis>
    <label>{}</label>
</exemple>\n""",
    pl.col("premise"),
    pl.col("hypothesis"),
    pl.col("label"),
)

# Extract a single example of each label
examples_str = (
    df_train.group_by("label")
    .head(1)
    .select(example_template)
    .to_series()
    .str.join()
    .item()
)

few_shot_prompt = (
    system_message
    + "Voici des exemples de classification corrects :\n"
    + examples_str
    + "Pour la nouvelle phrase, réponds uniquement par le nom de la relation (Conséquence, Contradiction, ou Neutre).\n"
    + "Réponds maintenant pour cet exemple précis :"
)

cot_prompt = (
    system_message
    + "Voici des exemples de classification corrects :\n"
    + examples_str
    + "Instructions pour la nouvelle tâche :\n"
    + "1. Analysez d'abord la logique étape par étape sous la balise <raisonnement>.\n"
    + "2. Conclus ensuite uniquement par le nom de la relation sous la balise <label>.\n"
    + "Format attendu :\n"
    + "<raisonnement>...ton analyse...</raisonnement>\n"
    + "<label>Conséquence OR Contradiction OR Neutre</label>"
)
print(cot_prompt)

Tu es un expert en logique et linguistique française. Ta tâche est de déterminer la relation entre une 'Prémisse' et une 'Hypothèse'.

Utilise strictement les définitions suivantes :
- Conséquence : L'hypothèse découle nécessairement de la prémisse.
- Contradiction : L'hypothèse est impossible si la prémisse est vraie.
- Neutre : La vérité de la prémisse ne permet pas de déterminer celle de l'hypothèse.

Voici des exemples de classification corrects :
<exemple>
    <premise>Eh bien, je ne pensais même pas à cela, mais j'étais si frustré, et j'ai fini par lui reparler.</premise>
    <hypothesis>Nous avons eu une grande discussion.</hypothesis>
    <label>Neutre</label>
</exemple>
<exemple>
    <premise>Eh bien, je ne pensais même pas à cela, mais j'étais si frustré, et j'ai fini par lui reparler.</premise>
    <hypothesis>Je ne lui ai pas parlé de nouveau</hypothesis>
    <label>Contradiction</label>
</exemple>
<exemple>
    <premise>Eh bien, je ne pensais même pas à cela, mais j'étais 

### Chargement du modèle (LLaMa 3.2 3B Instruct)

In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("HUGGING_FACE_HUB_TOKEN")
login(token=token)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"
DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device: ", DEVICE)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(MODEL_ID, dtype=DTYPE, device_map="auto")
model = torch.compile(model)

Using device:  cuda


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

### Fonction d'inférence

In [ ]:
def predict_batch(
    premise: list[str], hypotheses: list[str], system_prompt: str, batch_size=32
) -> list[str]:
    """
    Génère les prédictions à partir des prémises et hypothèses.
    """
    # Formatage des prompts
    prompts = [
        tokenizer.apply_chat_template(
            [
                {
                    "role": "user",
                    "content": f"{system_prompt}\n\n<premise>{p}</premise>\n<hypothesis>{h}</hypothesis>",
                }
            ],
            tokenize=False,
            add_generation_prompt=True,
        )
        for p, h in zip(premise, hypotheses)
    ]

    results = []
    for i in range(0, len(prompts), batch_size):
        batch = prompts[i : i + batch_size]

        inputs = tokenizer(
            batch, return_tensors="pt", padding=True, truncation=True
        ).to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=256,
                do_sample=False,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id,
            )

        # On garde uniquement les derniers tokens pour éviter de décoder le prompt à nouveau
        generated_ids = outputs[:, inputs.input_ids.shape[1] :]
        results.extend(tokenizer.batch_decode(generated_ids, skip_special_tokens=True))

    return results


def parse_response(response_col: str) -> pl.Expr:
    # Expression Regex pour trouver le label (insensible à la casse)
    label_pattern = r"(?i)\b(Conséquence|Contradiction|Neutre)\b"

    # On cherche directement le label
    return (
        pl.col(response_col)
        .str.extract(label_pattern, 1)
        .fill_null("Prédiction incorrecte")
    )


def parse_response_cot(response_col: str) -> pl.Expr:
    # Expression Regex pour trouver le label (insensible à la casse)
    label_pattern = r"(?i)\b(Conséquence|Contradiction|Neutre)\b"

    # Étape 1: On extrait le texte entre les tags <label>
    # Étape 2: On nettoie les espaces blancs et on cherche le label
    return (
        pl.col(response_col)
        .str.extract(r"(?i)<label>(.*?)</label>", 1)
        .str.strip_chars()
        .str.extract(label_pattern, 1)
        .fill_null("Prédiction incorrecte")
    )

### Jeu d'évaluation

In [6]:
sample_size = 200
# sample_size = df_val.height
df_val = df_val.head(sample_size)
f"Taille de l'ensemble d'évaluation: {df_val.height} éléments"

"Taille de l'ensemble d'évaluation: 200 éléments"

In [7]:
premises = df_val.get_column("premise").to_list()
hypotheses = df_val.get_column("hypothesis").to_list()
valid_labels = ["Neutre", "Conséquence", "Contradiction"]

### Zero-shot learning

In [ ]:
# Générer les prédictions
predictions = predict_batch(premises, hypotheses, zero_shot_prompt)

# Ajout et parsing des prédictions
df_val = df_val.with_columns(predictions_zero_shot=pl.Series(predictions)).with_columns(
    parse_response("predictions_zero_shot")
)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
# Prédictions non valides
valid = pl.col("predictions_zero_shot").is_in(valid_labels)

print(
    f"Nombre de prédictions invalides: ",
    df_val.filter(~valid).height,
    "/",
    df_val.height,
)
df_val.filter(~valid)

Nombre de prédictions invalides:  0 / 200


premise,hypothesis,label,predictions_zero_shot
str,str,str,str


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

print(
    classification_report(
        df_val.get_column("label"),
        df_val.get_column("predictions_zero_shot"),
        zero_division=np.nan,
    )
)

               precision    recall  f1-score   support

  Conséquence        nan      0.00      0.00        67
Contradiction       0.34      0.85      0.48        66
       Neutre       0.33      0.16      0.22        67

     accuracy                           0.34       200
    macro avg       0.33      0.34      0.23       200
 weighted avg       0.33      0.34      0.23       200



### Few shots learning

In [ ]:
# Générer les prédictions
predictions = predict_batch(premises, hypotheses, few_shot_prompt)

# Ajout et parsing des prédictions
df_val = df_val.with_columns(predictions_few_shot=pl.Series(predictions)).with_columns(
    parse_response("predictions_few_shot")
)

In [ ]:
# Prédictions non valides
valid = pl.col("predictions_few_shot").is_in(valid_labels)

print(
    f"Nombre de prédictions invalides: ",
    df_val.filter(~valid).height,
    "/",
    df_val.height,
)
df_val.filter(~valid)

Nombre de prédictions invalides:  0 / 200


premise,hypothesis,label,predictions_zero_shot,predictions_few_shot
str,str,str,str,str


In [ ]:
print(
    classification_report(
        df_val.get_column("label"),
        df_val.get_column("predictions_few_shot"),
        zero_division=np.nan,
    )
)

               precision    recall  f1-score   support

  Conséquence       0.34      1.00      0.50        67
Contradiction        nan      0.00      0.00        66
       Neutre        nan      0.00      0.00        67

     accuracy                           0.34       200
    macro avg       0.34      0.33      0.17       200
 weighted avg       0.34      0.34      0.17       200



### Chain-of-thoughts (CoT)

In [ ]:
# Générer les prédictions
predictions = predict_batch(premises, hypotheses, cot_prompt)

# Ajout et parsing des prédictions
df_val = df_val.with_columns(
    predictions_cot_raisonnement=pl.Series(predictions)
).with_columns(predictions_cot=parse_response_cot("predictions_cot_raisonnement"))

In [ ]:
# Prédictions non valides
valid = pl.col("predictions_cot").is_in(valid_labels)

print(
    f"Nombre de prédictions invalides: ",
    df_val.filter(~valid).height,
    "/",
    df_val.height,
)
df_val.filter(~valid).select(
    [
        "premise",
        "hypothesis",
        "label",
        "predictions_cot_raisonnement",
        "predictions_cot",
    ]
)

Nombre de prédictions invalides:  23 / 200


premise,hypothesis,label,predictions_cot_raisonnement,predictions_cot
str,str,str,str,str
"""Je ne savais pas dans quoi je …","""Je n'étais pas tout à fait cer…","""Conséquence""","""<raisonnement> La prémisse ind…","""Prédiction incorrecte"""
"""Et Grand-Mère racontait souven…","""La sœur de grand-mère n'était …","""Neutre""","""<raisonnement> La prémisse ind…","""Prédiction incorrecte"""
"""Et il était un coureur de jupo…","""Il était si fidèle et gentil.""","""Contradiction""","""<raisonnement> La première éta…","""Prédiction incorrecte"""
"""Et il était un coureur de jupo…","""Je le détestais parce qu'il ét…","""Neutre""","""<raisonnement> La première éta…","""Prédiction incorrecte"""
"""Et il était un coureur de jupo…","""Je n'étais pas un de ses fans.""","""Conséquence""","""<raisonnement> La première éta…","""Prédiction incorrecte"""
…,…,…,…,…
"""Euh, et donc ils ont juste qui…","""Elle a déménagé au Texas et el…","""Neutre""","""<raisonnement> La première éta…","""Prédiction incorrecte"""
"""Donc Granny s'est levé et elle…","""Mamie est restée sur le porche…","""Contradiction""","""<raisonnement> La prémisse ind…","""Prédiction incorrecte"""
"""Je ne l'ai jamais vu, et je ne…","""Je ne l'ai pas vu.""","""Conséquence""","""<raisonnement> La prémisse ind…","""Prédiction incorrecte"""


In [ ]:
print(
    classification_report(
        df_val.get_column("label"),
        df_val.get_column("predictions_cot"),
        zero_division=np.nan,
    )
)

                       precision    recall  f1-score   support

          Conséquence       0.41      0.58      0.48        67
        Contradiction       0.55      0.47      0.51        66
               Neutre       0.44      0.18      0.26        67
Prédiction incorrecte       0.00       nan      0.00         0

             accuracy                           0.41       200
            macro avg       0.35      0.41      0.31       200
         weighted avg       0.47      0.41      0.42       200



### Échantillons des prédictions

In [17]:
df_val.sample(10)

premise,hypothesis,label,predictions_zero_shot,predictions_few_shot,predictions_cot_raisonnement,predictions_cot
str,str,str,str,str,str,str
"""Et j'étais comme OK, et c'étai…","""après que j'ai dit oui, c’étai…","""Conséquence""","""Contradiction""","""Conséquence""","""<raisonnement> La prémisse ind…","""Conséquence"""
"""la chose était, il y avaient 1…","""Nous avons dû démonter les jet…","""Neutre""","""Neutre""","""Conséquence""","""<raisonnement> La prémisse ind…","""Conséquence"""
"""Ils ont fini par se retrouver …","""Il est resté à Brooklyn avec s…","""Neutre""","""Contradiction""","""Conséquence""","""<raisonnement> La prémisse ind…","""Conséquence"""
"""la chose était, il y avaient 1…","""Nous avons dû le démonter et l…","""Conséquence""","""Neutre""","""Conséquence""","""<raisonnement> La prémisse ind…","""Contradiction"""
"""Il n'a pas pu y aller.""","""Il n'avait pas l'autorisation …","""Conséquence""","""Contradiction""","""Conséquence""","""<raisonnement> La prémisse ind…","""Contradiction"""
"""J'ai besoin que tu fasses quel…","""Il y a quelque chose que j'ai …","""Conséquence""","""Contradiction""","""Conséquence""","""<raisonnement> La prémisse ind…","""Conséquence"""
"""C'est un peu unique en ce sens…","""Je n'ai jamais eu de travail.""","""Contradiction""","""Contradiction""","""Conséquence""","""<raisonnement> La prémisse ind…","""Contradiction"""
"""C'est vrai, c'était une journé…","""Cette journée m'a fait vraimen…","""Conséquence""","""Contradiction""","""Conséquence""","""<raisonnement> La prémisse ind…","""Neutre"""
"""Et ensuite, une fois que vous …","""Vous pourrez continuer lorsque…","""Neutre""","""Contradiction""","""Conséquence""","""<raisonnement> La prémisse est…","""Conséquence"""
